In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.csv('../data/calculated/final.csv', header = True, inferSchema = True)
df = df.withColumn('celebration', df.celebration.cast(StringType()))
df = df.drop("_c0")
df = df.drop('day')
df = df.drop("total_count")
df.printSchema()

df.show(1)

root
 |-- day_range: integer (nullable = true)
 |-- weather: string (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- celebration: string (nullable = true)
 |-- season: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_year: integer (nullable = true)
 |-- count_discretized: double (nullable = true)

+---------+-------+-----------+-----------+------+-----+-----------+-----------------+
|day_range|weather|day_of_week|celebration|season|month|day_of_year|count_discretized|
+---------+-------+-----------+-----------+------+-----+-----------+-----------------+
|     1200|  Sunny|          5|      false|Autumn|   12|         10|              2.0|
+---------+-------+-----------+-----------+------+-----+-----------+-----------------+
only showing top 1 row



In [3]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
day_range,1200,0,1200,600,1200
weather,Sunny,Partly cloudy,Sunny,Light rain shower,Partly cloudy
day_of_week,5,3,2,6,7
celebration,false,false,false,false,false
season,Autumn,Autumn,Autumn,Autumn,Summer
month,12,11,10,10,9
day_of_year,10,24,19,2,12
count_discretized,2,0,2,2,1


In [4]:
numeric_features = [t[0] for t in df.dtypes if ((t[1] == 'int') | (t[1] == 'double'))]
print("numeric_features")
print(numeric_features)
other_features = list(set(df.columns) - set(numeric_features))
print("other_features")
print(other_features)
categoricalColumns = ['day_range', 'weather', 'season', 'day_of_week', 'celebration', 'month', 'day_of_year']
print("categoricalColumns")
print(categoricalColumns)

numeric_features
['day_range', 'day_of_week', 'month', 'day_of_year', 'count_discretized']
other_features
['celebration', 'weather', 'season']
categoricalColumns
['day_range', 'weather', 'season', 'day_of_week', 'celebration', 'month', 'day_of_year']


# INUTILIZZATO

numeric_data = df.select(numeric_features).toPandas()

axs = pd.plotting.scatter_matrix(numeric_data, figsize=(8, 8));
n = len(numeric_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

In [5]:
sampled_data = df.select(numeric_features).sample(False, 0.8).toPandas()
axs = pd.plotting.scatter_matrix(sampled_data, figsize=(10, 10))
n = len(sampled_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

In [6]:
import six
for i in numeric_features:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to count_discretized for", i, df.stat.corr('count_discretized',i))

Correlation to count_discretized for day_range 0.453728249353354
Correlation to count_discretized for day_of_week 0.03944455803815452
Correlation to count_discretized for month -0.049039840626254064
Correlation to count_discretized for day_of_year 0.019198640558250425
Correlation to count_discretized for count_discretized 1.0


In [7]:
cols = df.columns
df.printSchema()
print(cols)

root
 |-- day_range: integer (nullable = true)
 |-- weather: string (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- celebration: string (nullable = true)
 |-- season: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_year: integer (nullable = true)
 |-- count_discretized: double (nullable = true)

['day_range', 'weather', 'day_of_week', 'celebration', 'season', 'month', 'day_of_year', 'count_discretized']


In [8]:
# da qui https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a

In [9]:
# suo da qui https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

print(categoricalColumns)
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
print("-----")    
print(stages)

label_stringIdx = StringIndexer(inputCol = 'count_discretized', outputCol = 'label')
stages += [label_stringIdx]
print(numeric_features)
numericCols = numeric_features
if 'count_discretized' in numericCols:
    numericCols.remove('count_discretized')
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
print("---", assemblerInputs)
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]
print("-----")
print(stages)

['day_range', 'weather', 'season', 'day_of_week', 'celebration', 'month', 'day_of_year']
-----
[StringIndexer_493bb6d96545b7f125b5, OneHotEncoderEstimator_42339473f487ba4ece0d, StringIndexer_419a89ac0edef0a11afd, OneHotEncoderEstimator_4ce39fc53c3796c3d0fe, StringIndexer_4674ac7970d178e86e72, OneHotEncoderEstimator_469ca0b58721007936ed, StringIndexer_4d92b1c5770c6db1695b, OneHotEncoderEstimator_40e6b1b8c6744995c348, StringIndexer_4ae79679709decccf542, OneHotEncoderEstimator_4f73bb8493c27e380839, StringIndexer_4873882aad8cd662dbd1, OneHotEncoderEstimator_48f2aa8f124972b9a2cd, StringIndexer_409d88776755f82aed71, OneHotEncoderEstimator_41d7a49b71418dc18919]
['day_range', 'day_of_week', 'month', 'day_of_year', 'count_discretized']
--- ['day_rangeclassVec', 'weatherclassVec', 'seasonclassVec', 'day_of_weekclassVec', 'celebrationclassVec', 'monthclassVec', 'day_of_yearclassVec', 'day_range', 'day_of_week', 'month', 'day_of_year']
-----
[StringIndexer_493bb6d96545b7f125b5, OneHotEncoderEstima

In [10]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- day_range: integer (nullable = true)
 |-- weather: string (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- celebration: string (nullable = true)
 |-- season: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_year: integer (nullable = true)
 |-- count_discretized: double (nullable = true)



In [11]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
label,0,1,0,0,2
features,"(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
day_range,1200,0,1200,600,1200
weather,Sunny,Partly cloudy,Sunny,Light rain shower,Partly cloudy
day_of_week,5,3,2,6,7
celebration,false,false,false,false,false
season,Autumn,Autumn,Autumn,Autumn,Summer
month,12,11,10,10,9
day_of_year,10,24,19,2,12
count_discretized,2,0,2,2,1


In [12]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 1039
Test Dataset Count: 419


In [13]:
# Decision tree
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label')
dtModel = dt.fit(train)
dtPredictions = dtModel.transform(test)
# predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)
# predictions.show(10)
dtPredictions.select('label', 'prediction', 'rawPrediction', 'probability').show(test.count())
print(test.count())

dtPredictions.filter('label != prediction').select('label', 'prediction', 'rawPrediction', 'probability').show(test.count())

+-----+----------+----------------+--------------------+
|label|prediction|   rawPrediction|         probability|
+-----+----------+----------------+--------------------+
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  0.0|       0.0|[291.0,7.0,44

+-----+----------+----------------+--------------------+
|label|prediction|   rawPrediction|         probability|
+-----+----------+----------------+--------------------+
|  0.0|       2.0|  [1.0,1.0,51.0]|[0.01886792452830...|
|  0.0|       2.0|  [1.0,1.0,51.0]|[0.01886792452830...|
|  0.0|       2.0|  [1.0,3.0,30.0]|[0.02941176470588...|
|  0.0|       2.0|  [1.0,3.0,30.0]|[0.02941176470588...|
|  0.0|       2.0|  [1.0,1.0,51.0]|[0.01886792452830...|
|  0.0|       2.0|  [1.0,3.0,30.0]|[0.02941176470588...|
|  1.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  1.0|       2.0|  [0.0,1.0,25.0]|[0.0,0.0384615384...|
|  1.0|       2.0|  [0.0,9.0,29.0]|[0.0,0.2368421052...|
|  1.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  1.0|       2.0|  [0.0,9.0,29.0]|[0.0,0.2368421052...|
|  1.0|       0.0|[291.0,7.0,44.0]|[0.85087719298245...|
|  1.0|       2.0|  [0.0,9.0,29.0]|[0.0,0.2368421052...|
|  1.0|       2.0|  [1.0,1.0,51.0]|[0.01886792452830...|
|  1.0|       0.0| [45.0,2.0,43

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator()
evaluator.setMetricName("accuracy")
print("Accuracy", evaluator.evaluate(dtPredictions))
print(evaluator.getMetricName())
evaluator.setMetricName("weightedPrecision")
print("weightedPrecision", evaluator.evaluate(dtPredictions))
print(evaluator.getMetricName())
evaluator.setMetricName("weightedRecall")
print("weightedRecall", evaluator.evaluate(dtPredictions))
print(evaluator.getMetricName())
evaluator.setMetricName("f1")
print("f1", evaluator.evaluate(dtPredictions))
print(evaluator.getMetricName())

Accuracy 0.863961813842482
accuracy
weightedPrecision 0.8708695780801485
weightedPrecision
weightedRecall 0.863961813842482
weightedRecall
f1 0.8617882187209845
f1


In [15]:
dtModel.toDebugString

'DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4208869d339a56f88faa) of depth 5 with 49 nodes\n  If (feature 72 <= 300.0)\n   If (feature 25 in {0.0})\n    If (feature 65 in {0.0})\n     If (feature 30 in {0.0})\n      If (feature 29 in {0.0})\n       Predict: 1.0\n      Else (feature 29 not in {0.0})\n       Predict: 2.0\n     Else (feature 30 not in {0.0})\n      Predict: 1.0\n    Else (feature 65 not in {0.0})\n     If (feature 37 in {0.0})\n      Predict: 1.0\n     Else (feature 37 not in {0.0})\n      Predict: 2.0\n   Else (feature 25 not in {0.0})\n    If (feature 47 in {0.0})\n     If (feature 57 in {0.0})\n      If (feature 59 in {0.0})\n       Predict: 1.0\n      Else (feature 59 not in {0.0})\n       Predict: 2.0\n     Else (feature 57 not in {0.0})\n      Predict: 2.0\n    Else (feature 47 not in {0.0})\n     Predict: 2.0\n  Else (feature 72 > 300.0)\n   If (feature 2 in {0.0})\n    If (feature 27 in {0.0})\n     If (feature 25 in {0.0})\n      If (feature 32 i

In [16]:
# BAYES

from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
nbModel = nb.fit(train)

# select example rows to display.
nbPredictions = nbModel.transform(test)
nbPredictions.select("label", "prediction").show()

nbPredictions.filter('label != prediction').select('label', 'prediction', 'rawPrediction', 'probability').show(test.count())

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows

+-----+----------+--------------------+--------------------+
|label|prediction|       rawPrediction|         probability|
+-----+----------+--------------------+--------------------+
|  0.0|       1.0|[-251.15303218818...|[1.62800738475567...|
|  0.0|       1.0|[-231.55827038751...|[0.00276390831615...|
|  0.0|       1.0|[-252.23706411849...|[4.84897742437215...|
|  0.0|       1.0|[-244.05455668163...|[4.94338673750357...|
|  0.0|       1.0|[-214.61773214712...|[0.41058180652387...|
|  0.0|       1.0|[-244.056248

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
nbEvaluator = MulticlassClassificationEvaluator()
nbEvaluator.setMetricName("accuracy")
print("Accuracy", nbEvaluator.evaluate(nbPredictions))
print(nbEvaluator.getMetricName())
nbEvaluator.setMetricName("weightedPrecision")
print("weightedPrecision", nbEvaluator.evaluate(nbPredictions))
print(nbEvaluator.getMetricName())
nbEvaluator.setMetricName("weightedRecall")
print("weightedRecall", nbEvaluator.evaluate(nbPredictions))
print(nbEvaluator.getMetricName())
nbEvaluator.setMetricName("f1")
print("f1", nbEvaluator.evaluate(nbPredictions))
print(nbEvaluator.getMetricName())

Accuracy 0.6945107398568019
accuracy
weightedPrecision 0.7228569353065616
weightedPrecision
weightedRecall 0.6945107398568019
weightedRecall
f1 0.6878570297292862
f1
